In [0]:
# ---------------------------------------------------------
# 02_Silver_Cleaning (v2 - Safe Mode)
# Goal: Clean raw data and create the 'default_flag' target
# ---------------------------------------------------------

from pyspark.sql.functions import col, when, regexp_replace

# 1. Load Bronze Data
print("⏳ Loading Bronze Table...")
df = spark.table("bronze_lending_club")

# 2. Filter: Keep only Completed Loans
print("🧹 Filtering for 'Fully Paid' or 'Charged Off'...")
df_silver = df.filter(col("loan_status").isin("Fully Paid", "Charged Off"))

# 3. Create Target Variable (1 = Default, 0 = Paid)
df_silver = df_silver.withColumn(
    "default_flag", 
    when(col("loan_status") == "Charged Off", 1).otherwise(0)
)

# 4. Clean Feature: 'term' (Remove " months" and convert to Int)
df_silver = df_silver.withColumn(
    "term_clean", 
    regexp_replace(col("term"), " months", "").cast("int")
)

# 5. Clean Feature: 'emp_length' (Extract numbers, e.g. "10+ years" -> 10)
df_silver = df_silver.withColumn(
    "emp_length_clean", 
    when(
        regexp_replace(col("emp_length"), "[^0-9]", "") == "", 
        None
    ).otherwise(
        regexp_replace(col("emp_length"), "[^0-9]", "").cast("int")
    )
)

# 6. Select Columns (REMOVED 'fico_range_low' to fix error)
cols_to_keep = [
    "loan_amnt",            # Amount requested
    "int_rate",             # Interest Rate
    "installment",          # Monthly payment
    "annual_inc",           # Income
    "dti",                  # Debt-to-Income Ratio
    "term_clean",           # Loan duration
    "emp_length_clean",     # Stability
    "home_ownership",       # Collateral
    "purpose",              # Reason for loan
    "addr_state",           # Geography
    "default_flag"          # Target
]

# Drop rows with missing values
df_final = df_silver.select(cols_to_keep).dropna()

# 7. Save to Silver Delta Table
table_name = "silver_lending_club"
print(f"💾 Saving to Silver Table: {table_name}...")
df_final.write.format("delta").mode("overwrite").saveAsTable(table_name)

# --- VALIDATION ---
print("-" * 30)
print(f"🚀 SUCCESS! Silver Table Created.")
print(f"📊 Rows after cleaning: {spark.table(table_name).count():,}")
print(f"💀 Default Rate: {df_final.filter('default_flag = 1').count() / df_final.count():.2%}")
print("-" * 30)
display(spark.table(table_name).limit(5))